In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import null_space

In [3]:
def generate_matrix_with_rank(k, n, r):
    if r > min(k, n):
        raise ValueError("Rank r cannot be greater than min(k, n)")

    # Step 1: Generate a k x r matrix with rank r
    # This is done by generating a random matrix and using QR decomposition to ensure it has rank r.
    A = np.random.rand(k, r)
    Q, R = np.linalg.qr(A)  # QR decomposition
    A = Q[:, :r]  # Ensure the matrix has rank r

    # Step 2: Extend to a k x n matrix while preserving rank r
    if n > r:
        # Generate a random r x (n-r) matrix
        B = np.random.rand(r, n-r)
        # Multiply to get the additional columns
        additional_columns = np.dot(A, B)
        # Concatenate to get the final k x n matrix
        final_matrix = np.hstack((A, additional_columns))
    else:
        final_matrix = A

    return final_matrix

In [164]:
np.random.seed(1)

# Parameters
n = 50
m = 25
k = 200
rF = 23
rR = n-2

# Gen matrices
F = generate_matrix_with_rank(m, n, rF)
R = generate_matrix_with_rank(k, n, rR)

# Check common kernel
A = np.vstack([F, R])
assert np.linalg.matrix_rank(A) == n, "common kernel condition not satisfied"

b1 = np.random.normal(size=m)
b2 = np.random.normal(size=k)
bstack = np.hstack([b1, b2])

# Get solution from original problem form
x_original_sol, _, _, _ = np.linalg.lstsq( A, bstack )

Rpinv = np.linalg.pinv(R)
W = null_space(R)
FWpinv = np.linalg.pinv(F @ W)
xker = W @ FWpinv @ b1

G = np.eye(n) - ( W @ FWpinv @ F )
H = F @ G
Mpinv = np.linalg.pinv(R.T @ R)
Ropinv = G @ Rpinv 

# Get solution via transformed method
Atilde = np.vstack([F @ Ropinv, np.eye(k)])
bt = np.hstack([b1, R @ Rpinv @ b2])
w_original_sol, _, _, _ = np.linalg.lstsq( Atilde, bt)
x_transformed_sol = xker + (Ropinv @ w_original_sol)


/var/folders/fn/xbs7g2ln57x3xrdggzf_dzzm0000gr/T/ipykernel_7150/2661507187.py:23: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x_original_sol, _, _, _ = np.linalg.lstsq( A, bstack )
/var/folders/fn/xbs7g2ln57x3xrdggzf_dzzm0000gr/T/ipykernel_7150/2661507187.py:38: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_original_sol, _, _, _ = np.linalg.lstsq( Atilde, bt)


In [165]:

# CGLS initializations
#x = np.zeros(n) + 1000
x = 100* (Ropinv @ w_original_sol)
#x = Ropinv @ w_original_sol
#x = (Ropinv @ w_original_sol) + 1e0*np.random.normal(size=Ropinv.shape[0]) 
btilde2 = Mpinv @ R.T @ b2
d = np.hstack([ b1 - F @ x , (R @ btilde2) - (R @ x)  ])
u = Mpinv @ H.T @ d[:m] 
rprev = (R @ u) + d[m:]
p = rprev
qprev = G @ (u + btilde2) - x
qhat = qprev
z = np.hstack([F @ qhat, p])

#print(np.linalg.norm(rprev))

#roriginal = np.linalg.norm( A.T @ ( A @ (x + xker) - bstack  ) )
#print(roriginal)


#print( np.linalg.norm( Atilde.T @ ( (Atilde @ R @ x) - bt   )   ) )
#print()


w = R @ x
_d = bt - Atilde @ w
_rprev = Atilde.T @ _d
_p = _rprev
_z = Atilde @ _p


#print(np.linalg.norm(d - _d))
print(np.linalg.norm(xker + x - x_original_sol))


for j in range(k):

    alpha = (np.linalg.norm(rprev)/np.linalg.norm(z))**2
    x += alpha*qhat
    d -= alpha*z
    u = Mpinv @ H.T @ d[:m]
    rnext = (R @ u) + d[m:]
    #print(np.linalg.norm(rnext))
    
    #roriginal = np.linalg.norm( A.T @ ( A @ (x + xker) - bstack  ) )
    #print(roriginal)

    
    #print( np.linalg.norm( Atilde.T @ ( (Atilde @ R @ x) - bt   )   ) )
    #print()

    beta = (np.linalg.norm(rnext)/np.linalg.norm(rprev))**2
    p = rnext + beta*p
    qnext = G @ (u + btilde2) - x
    qhat = qnext + (beta - alpha)*qprev
    z = np.hstack([F @ qhat, p])

    #print(np.linalg.norm(xker + x - x_original_sol))

    _alpha = (np.linalg.norm( _rprev )/np.linalg.norm( _z ))**2
    w = w + _alpha*_p
    _d -= _alpha*_z
    _rnext = Atilde.T @ _d
    _beta = (np.linalg.norm(_rnext)/np.linalg.norm(_rprev))**2
    _p = _rnext + _beta*_p
    _z = Atilde @ _p

    #print(np.linalg.norm(d - _d))
    print(np.linalg.norm(xker + x - x_original_sol))

    # Update old parts
    rprev = rnext.copy()
    qprev = qnext.copy()

    _rprev = _rnext.copy()

x_ecgls_sol = xker + x


597.1850721563752
376.7149817483336
280.75037775233744
208.31470899995742
147.5179591667544
117.38969941444583
80.60336433109832
60.5592644762483
48.64634073073495
34.00851891073934
25.17648971581693
19.220981146177017
14.556801673270753
10.73478477534099
8.144116683158355
6.13398340175412
4.690733377925876
3.4874011637711306
2.626834752321474
1.9701639364832206
1.4917757438294839
1.1255023826294355
0.854338040976825
0.6472266514706843
0.4890604970057748
0.3707602957705069
0.27971360884251417
0.212150227834809
0.15997181686047
0.12139909900452492
0.09151845480523085
0.06947515146478955
0.05237944493823949
0.03976392386860191
0.02999344568053278
0.02276270020792432
0.01718373672922895
0.01303365343854119
0.009849479160536623
0.007465090152572142
0.005647420635261508
0.004276905643806355
0.0032385137121687183
0.002450935154808224
0.0018570887324401333
0.0014047935005871658
0.0010648191344074236
0.0008052726281430314
0.0006104752780660297
0.0004616398160910668
0.0003499593175850782
0.0002

In [155]:
np.linalg.norm( x_original_sol - x_ecgls_sol )

7.665585092849123e-08

In [152]:
np.linalg.norm(x_original_sol - x_transformed_sol)

4.3225895825118735e-14

# Is this being caused by the condition number of prior?

In [167]:
import jlinops

In [242]:
np.random.seed(7)

# Parameters
n = 50
m = 25
rF = 23


# Gen matrices
F = generate_matrix_with_rank(m, n, rF)
#R = generate_matrix_with_rank(k, n, rR)
R, W = jlinops.first_order_derivative_1d(n, boundary="reflexive")
R = R.toarray()
k = R.shape[0]

# Check common kernel
A = np.vstack([F, R])
assert np.linalg.matrix_rank(A) == n, "common kernel condition not satisfied"

b1 = np.random.normal(size=m)
b2 = np.random.normal(size=k)
bstack = np.hstack([b1, b2])

# Get solution from original problem form
x_original_sol, _, _, _ = np.linalg.lstsq( A, bstack )

Rpinv = np.linalg.pinv(R)
W = null_space(R)
FWpinv = np.linalg.pinv(F @ W)
xker = W @ FWpinv @ b1

G = np.eye(n) - ( W @ FWpinv @ F )
H = F @ G
Mpinv = np.linalg.pinv(R.T @ R)
Ropinv = G @ Rpinv 

# Get solution via transformed method
Atilde = np.vstack([F @ Ropinv, np.eye(k)])
bt = np.hstack([b1, R @ Rpinv @ b2])
w_original_sol, _, _, _ = np.linalg.lstsq( Atilde, bt)
x_transformed_sol = xker + (Ropinv @ w_original_sol)

/var/folders/fn/xbs7g2ln57x3xrdggzf_dzzm0000gr/T/ipykernel_7150/1564031249.py:25: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x_original_sol, _, _, _ = np.linalg.lstsq( A, bstack )
/var/folders/fn/xbs7g2ln57x3xrdggzf_dzzm0000gr/T/ipykernel_7150/1564031249.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_original_sol, _, _, _ = np.linalg.lstsq( Atilde, bt)


In [243]:
# # Check if identities hold
# #w = R @ (100*np.random.uniform(size=R.shape[1]))
# w = 100*np.random.uniform(size=R.shape[0])
# x = Ropinv @ w
# np.linalg.norm(w - R @ x)

In [244]:
# x = Ropinv @ R @ (100*np.random.uniform(size=R.shape[1]))
# np.linalg.norm( x - Ropinv @ R @ x )

In [248]:

# CGLS initializations
# x = 10 + np.random.normal(size=Ropinv.shape[0]) 
#x = np.zeros(n) + 1000
# x = 100 * (Ropinv @ w_original_sol)
#x = Ropinv @ w_original_sol
x = (Ropinv @ ( w_original_sol + 1e0*np.random.normal(size=Ropinv.shape[0]) ) ) 

btilde2 = Mpinv @ R.T @ b2

d = np.hstack([ b1 - F @ x , (R @ btilde2) - (R @ x)  ])
u = Mpinv @ H.T @ d[:m] 
rprev = (R @ u) + d[m:]
d2tilde = (G @ btilde2) - x
ptilde = (G @ u) + d2tilde 
p = rprev
z = np.hstack([F @ ptilde, p])

#print(np.linalg.norm(rprev))

#roriginal = np.linalg.norm( A.T @ ( A @ (x + xker) - bstack  ) )
#print(roriginal)

#print( np.linalg.norm( Atilde.T @ ( (Atilde @ R @ x) - bt   )   ) )
#print()


w = R @ x
_d = bt - Atilde @ w
_rprev = Atilde.T @ _d
_p = _rprev
_z = Atilde @ _p


#print(np.linalg.norm(d - _d))
#print(np.linalg.norm(rprev - _rprev))
print(np.linalg.norm(p - _p))
#print(np.linalg.norm(z - _z))
#print(np.linalg.norm(xker + x - x_original_sol))


for j in range(k*10):

    alpha = (np.linalg.norm(rprev)/np.linalg.norm(z))**2
    x += alpha*ptilde
    d -= alpha*z
    u = Mpinv @ H.T @ d[:m]
    rnext = (R @ u) + d[m:]

    beta = (np.linalg.norm(rnext)/np.linalg.norm(rprev))**2
    p = rnext + beta*p
    d2tilde -= alpha*ptilde
    ptilde = (G @ u) + d2tilde + beta*ptilde
    z = np.hstack([F @ ptilde, p])

    #print(np.linalg.norm(xker + x - x_original_sol))

    _alpha = (np.linalg.norm( _rprev )/np.linalg.norm( _z ))**2
    w = w + _alpha*_p
    _d -= _alpha*_z
    _rnext = Atilde.T @ _d
    _beta = (np.linalg.norm(_rnext)/np.linalg.norm(_rprev))**2
    _p = _rnext + _beta*_p
    _z = Atilde @ _p


    print(np.linalg.norm(p - _p))
    #print(np.linalg.norm(z - _z))

    #print(np.linalg.norm(d - _d))
    #print(np.linalg.norm(xker + x - x_original_sol))

    # Update old parts
    rprev = rnext.copy()
    qprev = qnext.copy()

    _rprev = _rnext.copy()


x_ecgls_sol = xker + x


2.7532336746379607e-12
1.0002807313561664e-12
2.19851569071016e-12
5.485539219011691e-12
1.9156656504182607e-11
3.1514327102418913e-10
5.598479782227189e-09
1.2060831676724914e-07
2.254820531992268e-06
3.7652650507835245e-05
0.0008709790183976943
0.01487362935447134
0.8924484405997963
0.015357177085036477
0.0004909753044715849
0.009936559946354815
0.0029909411641507277
0.0002698726094033711
0.00014803351286108687
0.0001841343090702842
6.28357030098625e-06
1.3492244078636537e-06
1.2154002953285642e-05
4.378916326764547e-10
9.363394182521542e-09
1.1942631682300644e-07
2.4950014505336765e-09
3.2624733765524497e-09
1.643552602353446e-09
6.13303235176664e-10
2.0241721095360136e-10
8.503427047506257e-11
2.923775181138677e-11
9.31341559967512e-13
6.996504665540041e-13
2.9179147952931627e-14
6.941897150535235e-14
1.0770174702030337e-13
4.0117791237411764e-14
1.0734321121143962e-13
3.893690931048303e-14
6.027063466791378e-14
6.466389282104353e-14
2.430713794700074e-13
1.4570064581906747e-13
1.3

In [246]:

# # CGLS initializations
# x = np.zeros(n)
# #x = np.zeros(n) + 1000
# # x = 100 * (Ropinv @ w_original_sol)
# #x = Ropinv @ w_original_sol
# #x = (Ropinv @ w_original_sol) + 1e0*np.random.normal(size=Ropinv.shape[0]) 
# btilde2 = Mpinv @ R.T @ b2
# d = np.hstack([ b1 - F @ x , (R @ btilde2) - (R @ x)  ])
# u = Mpinv @ H.T @ d[:m] 
# rprev = (R @ u) + d[m:]
# p = rprev
# qprev = G @ (u + btilde2) - x
# qhat = qprev
# z = np.hstack([F @ qhat, p])

# #print(np.linalg.norm(rprev))

# #roriginal = np.linalg.norm( A.T @ ( A @ (x + xker) - bstack  ) )
# #print(roriginal)


# #print( np.linalg.norm( Atilde.T @ ( (Atilde @ R @ x) - bt   )   ) )
# #print()


# w = R @ x
# _d = bt - Atilde @ w
# _rprev = Atilde.T @ _d
# _p = _rprev
# _z = Atilde @ _p


# #print(np.linalg.norm(d - _d))
# #print(np.linalg.norm(rprev - _rprev))
# #print(np.linalg.norm(p - _p))
# #print(np.linalg.norm(z - _z))
# #print(np.linalg.norm(xker + x - x_original_sol))


# for j in range(k*10):

#     alpha = (np.linalg.norm(rprev)/np.linalg.norm(z))**2
#     x += alpha*qhat
#     d -= alpha*z
#     u = Mpinv @ H.T @ d[:m]
#     rnext = (R @ u) + d[m:]

#     #print(np.linalg.norm(rnext))
    
#     #roriginal = np.linalg.norm( A.T @ ( A @ (x + xker) - bstack  ) )
#     #print(roriginal)

#     #print( np.linalg.norm( Atilde.T @ ( (Atilde @ R @ x) - bt   )   ) )
#     #print()

#     beta = (np.linalg.norm(rnext)/np.linalg.norm(rprev))**2
#     p = rnext + beta*p
#     qnext = G @ (u + btilde2) - x0 # THIS SHOULD HAVE x0!
#     qhat = qnext + (beta - alpha)*qprev 
#     z = np.hstack([F @ qhat, p])

#     #print(np.linalg.norm(xker + x - x_original_sol))

#     _alpha = (np.linalg.norm( _rprev )/np.linalg.norm( _z ))**2
#     w = w + _alpha*_p
#     _d -= _alpha*_z
#     _rnext = Atilde.T @ _d
#     _beta = (np.linalg.norm(_rnext)/np.linalg.norm(_rprev))**2
#     _p = _rnext + _beta*_p
#     _z = Atilde @ _p

#     print(_d[m:])

#     #print(np.linalg.norm(p - _p))
#     #print(np.linalg.norm(z - _z))

#     #print(np.linalg.norm(d - _d))
#     #print(np.linalg.norm(xker + x - x_original_sol))

#     # Update old parts
#     rprev = rnext.copy()
#     qprev = qnext.copy()

#     _rprev = _rnext.copy()


# x_ecgls_sol = xker + x


In [206]:
np.linalg.cond(Atilde)

11.236906543091974

In [207]:
np.linalg.cond(A)

76.97550387072367